Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [1]:
from random import choices, randint, choice, sample, random
from copy import copy, deepcopy

import lab9_lib
import numpy as np
import math


In [2]:
fitness = lab9_lib.make_problem(1)
for n in range(10):     #Determina il numero di Fitness call
    ind = choices([0, 1], k=1000)     #genoma casuale di 50 elementi
    print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")

print(fitness.calls)

0001010110101010010000011101110100010100010110011101010101100001001011000100001100100110111100100110000110100100100001011111111111000010010010010100111000110000101101001100000010011100010100000111000011110001010111011100101111001111010001110001110011011101001010010100010010000101110100001110110011000110000010010000010100011001000001100101110101100100100111110111011100101000011000001011101100000110001101111111001001101110100101001111111101010010111011111010101101000100000111111010101010001111101111100011001101100000110111001010010000111001100010001101111011101100011100010011010100100010001011110110101110110000111101011001001000011110000110111000010001100100111100100110100010001000001111101101110111000111000011001010101111011011111010001101101000101110011011010110010001010001000100110001111011111000110100011101100010110101011100101010110100101001001000101111100001011100111100010101011001110111101111001101010101010100011100001010100001111010001101110011100100100110110100010000111011101010

# Local Search Algorithm

In [34]:
# GLOBAL PARAMETER #
NUM_LOCI = 1000

POPULATION_SIZE = 10
NEW_OFFSPRING = 30
TOURNAMENT_SIZE = 5
MUTATION_PROBABILITY = .15
NUM_ISLANDS = 3

# . . . . . . #

### Mutation Techniques

In [4]:
def mutation1(ind):     # 1 loci(gene) mutated
    offspring = copy(ind)
    pos = randint(0, NUM_LOCI-1)
    offspring[pos] = 1 - offspring[pos]     # Se ho T/F-> offspring[pos] = not offspring[pos]
    #assert len(offspring) == NUM_LOCI
    return offspring

def mutation2(ind):     # Reset randomly a random number of loci
    offspring = copy(ind)
    poss = (randint(0, NUM_LOCI-1), randint(0, NUM_LOCI-1))
    offspring = ind[:min(poss)] + [choice([0, 1])for _ in range(max(poss)-min(poss))] + ind[max(poss):]
    #assert len(offspring) == NUM_LOCI

    return offspring

### Recombination Techniques

In [5]:
def one_cut_crossover(ind1, ind2):
    cut_point = randint(0, NUM_LOCI-1)
    offspring = ind1[:cut_point] + ind2[cut_point:]
    #    assert len(offspring) == NUM_LOCI
    return offspring

'''def n_cut_crossover(ind1: fitness, ind2: fitness) -> fitness:
    num_cut = math.ceil(np.random.normal(loc=10, scale=5))
    cut_point = randint(0, NUM_LOCI-1)
    offspring = ind1[:(NUM_LOCI/num_cut)]

    offspring = ind1[:cut_point] + ind2[cut_point:]
    assert len(offspring.genotype) == NUM_LOCI
    return offspring'''

def uniform_crossover(ind1, ind2):
    offspring = [ind1[i] if i % 2 else ind2[i] for i in range(NUM_LOCI)]
    # assert len(offspring) == NUM_LOCI
    return offspring

### Parent Selection

In [30]:
def roulette_wheel(population):     # Roulette wheel with same probability
    return choice(population)

def variable_tournament(population):
    
    n_partecipants = randint(2, int(len(population)/2))
    pool = choices(population, k=n_partecipants)
    parent = max(pool, key=lambda i: fitness(i))

    return parent

def static_tournament(pop):

    pool = [choice(pop) for _ in range(TOURNAMENT_SIZE)]
    champ = max(pool, key=lambda i: fitness(i))
    
    return champ

def best_parent_ever(population):
    champ = max(population, key=lambda i: fitness(i))
    return champ

def genotype_screening(population):     # Select the 2 element with most differnt genotype 
    distance = []
    for i in range(len(population)):
        distance.append([sum(np.bitwise_xor(np.array(population[i]), np.array(population[j]))) for j in range(len(population))])
    
    distance = np.array(distance)
    max_position = np.unravel_index(np.argmax(distance, axis=None), distance.shape)
    
    return population[max_position[0]], population[max_position[1]]

### Survival Selection

In [7]:
def survival_selection(population):
    population.sort(key=lambda i:fitness(i), reverse=True)       # ORDERING FROM BEST TO WORSE
    return population[:POPULATION_SIZE]       # SURVIVAL SELECTION
    
def remove_twin(population):        # Remove TWIN from the population because I belive that they will have the same fitness
    twins = set()
    for i in range(len(population)):
        for j in range(i+1, len(population)):
            if population[i]==population[j]:
                twins.add(j)
    
    new_p = [ind for i, ind in enumerate(population) if i not in twins]
    return new_p



### Island Definition

In [8]:
def island_0(population):   #Standard screening type of island
    return #...

def island_1(population):   #Genotype screening type of island
    offspring = list()
    for counter in range(NEW_OFFSPRING):
        if random() < MUTATION_PROBABILITY:
            # MUTATION 
            p = roulette_wheel(population)
            o = mutation1(p)
        else:
            #CROSS-OVER
            p1 = variable_tournament(population)
            p2 = variable_tournament(population)
            o =  one_cut_crossover(p1, p2)
        offspring.append(o)

    population.extend(offspring)
    population = remove_twin(population)
    population = survival_selection(population)
    return population

def island_2(population):   # Fitness screening type of island
    return #...

def island_3(population):   # Where all the result come togheter
    return #...

## Problem start from HERE ->

In [41]:
fitness = lab9_lib.make_problem(1)

starting_population = [choices([0, 1], k=NUM_LOCI) for _ in range(POPULATION_SIZE)]

In [10]:
print(len(starting_population))

print(starting_population[1]==starting_population[9])

10
False


In [43]:
population = starting_population
island_0 = starting_population
champ_island=[]
galapagos = (island_0, champ_island)
mutation_strategy = (mutation1, mutation2)
recombination_strategy = (one_cut_crossover, uniform_crossover)
parent_selection_strategy = (roulette_wheel, static_tournament, variable_tournament)
survival_selection_strategy = (remove_twin, survival_selection)

while all(fitness(i) != 1 for i in population):     # condizione da sistemare
    # Island 0 #
    offspring = list()
    for counter in range(NEW_OFFSPRING):
        if random() < MUTATION_PROBABILITY:
            # MUTATION 
            p = roulette_wheel(population)
            o = mutation1(p)
        else:
            #CROSS-OVER
            p1, p2 = genotype_screening(population)
            #p1 = variable_tournament(population)
            #p2 = variable_tournament(population)
            o =  uniform_crossover(p1, p2)
        offspring.append(o)

    population.extend(offspring)
    population = remove_twin(population)
    population = survival_selection(population)
    
    # Island 1 #
    
    # Island 2 #


    print(fitness(population[0]))
print(fitness.calls)

0.52
0.521
0.522
0.523
0.524
0.524
0.525
0.526
0.527
0.527
0.527
0.527
0.527
0.527
0.528
0.529
0.53
0.531
0.532
0.532
0.532
0.532
0.533
0.534
0.534
0.534
0.535
0.535
0.535
0.535
0.536
0.536
0.537
0.538
0.538
0.538
0.538
0.539
0.539
0.54
0.54
0.54
0.54
0.541
0.541
0.541
0.541
0.542
0.542
0.542
0.542
0.543
0.543
0.543
0.543
0.545
0.545
0.546
0.546

KeyboardInterrupt: 